## Basic network monitor
This notebook implements a simple network monitor to analyse the remote
network connections and traffic of this machine

In [ ]:
# Imports
import time
import urllib3
import nbutils
import netmonitor.commons as com

In [ ]:
# Global settings
SECONDS_DELAY = 5
LOOK_UP_IPS = True

nbutils.set_default_pd_options()

### Connections

In [ ]:
# Get snapshot of network connections
con_frame = com.get_connection_snapshot()

In [ ]:
# Remove local IP addresses
con_frame = con_frame[~con_frame["ip_local"]].copy()

In [ ]:
# Look up process names
con_frame["process"] = con_frame["pid"].apply(com.get_process_name)

In [ ]:
# Look up IPs
if LOOK_UP_IPS and len(con_frame) > 0:
    pool_manager = urllib3.PoolManager()
    con_frame["ip_infos"] = con_frame["ip_address"].apply(
        com.get_ip_infos, args=(pool_manager,))
    con_frame["ip_org"], con_frame["ip_country"] = zip(*con_frame["ip_infos"])
else:
    con_frame["ip_org"] = ""*len(con_frame)
    con_frame["ip_country"] = ""*len(con_frame)

In [ ]:
# Print connections
cols = ["ip_address", "port", "ip_org", "ip_country", "pid", "process", "status"]
print(con_frame[cols])

### Traffic

#### Absolute

In [ ]:
# Get traffic snapshot
traffic_frame1 = com.get_traffic_snapshot()
traffic_frame1 = traffic_frame1.sort_values(by="network_interface")
print(traffic_frame1)

#### Diff

In [ ]:
# Get second traffic snapshot and calculate diff
print(f"Calculating diff in {SECONDS_DELAY} seconds...\n")
time.sleep(SECONDS_DELAY)

traffic_frame2 = com.get_traffic_snapshot()
traffic_frame2 = traffic_frame2.sort_values(by="network_interface")

diff_columns = ["packets_recv", "packets_sent", "bytes_recv", "bytes_sent"]
diff_frame = traffic_frame2[diff_columns] - traffic_frame1[diff_columns]

print_frame = traffic_frame1[["network_interface"]]
print_frame = print_frame.join(diff_frame)

print(print_frame)